In [2]:
"""
Bicicletas recorridos
Clima PRCP TAVG
2020-2021
acquisition clean prepare upload
"""
import os
import sys

import urllib.request
# from zipfile import ZipFile

import pandas as pd
import numpy as np

import random

# Mover esto a exploring
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib as mpl

# mpl.rcParams.update( mpl.rcParamsDefault ) 

In [3]:
bicicletas = {
    "recorridos" : {
        "2020" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
            "zip_file" : "./datasets/recorridos-realizados-2020.zip",
            "df" : pd.DataFrame(),
        },
        "2021" : {
            "url" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
            "zip_file" : "./datasets/recorridos-realizados-2021.zip",
            "df" : pd.DataFrame(),
        },
    },
    "recorridos_df" : pd.DataFrame(),
    "estaciones" : pd.DataFrame(),
}

In [4]:
"""
Descargar recorridios bicicletas
"""
def descargar_recoridos(
    recorridos,
    equivalencias_encabezados
):
    for r in recorridos:
        if not os.path.exists( recorridos[r]["zip_file"] ):
            print("Descargando", recorridos[r]["url"])
            urllib.request.urlretrieve(
              recorridos[r]["url"],
              recorridos[r]["zip_file"]
            )
        else:
            print("Ya descargado", recorridos[r]["zip_file"])  
        recorridos[r]["df"] = pd.read_csv(
            recorridos[r]["zip_file"],
            compression = 'zip',
            encoding = 'utf-8', 
        )
#         print( recorridos[r]["df"].info() )
    # ajustar 2021
    recorridos["2021"]["df"] = recorridos["2021"]["df"]
    recorridos["2021"]["df"] = recorridos["2021"]["df"].rename( columns = equivalencias_encabezados )
    recorridos["2021"]["df"]["id"] = recorridos["2021"]["df"]['id'].astype('Int64')
    print('done')
    return recorridos

equivalencias_encabezados = {
        'ID'                                  : 'id',
        'Duración'                            : 'duracion_recorrido',
        'Estado cerrado'                      : 'estado_cerrado',
        'ID de ciclista'                      : 'id_usuario',
        'Tipo de ciclista'                    : 'cilicsta_tipo',
        'Modelo de bicicleta'                 : 'bicicleta_modelo',   

        'Id de estación de inicio'            : 'id_estacion_origen', 
        'Nombre de estación de inicio'        : 'nombre_estacion_origen',
        'Fecha de inicio'                     : 'fecha_origen_recorrido',     
        'Origen de viaje'                     : 'direccion_estacion_origen',  
        # '':'long_estacion_origen',        
        # '':'lat_estacion_origen',
        'Id de estación de fin de viaje'      : 'id_estacion_destino',     
        'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
        'Fecha de fin'                        : 'fecha_destino_recorrido',          
        # '':'direccion_estacion_destino',           
        # '':'long_estacion_destino',       
        # '':'lat_estacion_destino',
        # '':'periodo',  
}
bicicletas["recorridos"] = descargar_recoridos(
    bicicletas["recorridos"],
    equivalencias_encabezados
)

ya descargado ./datasets/recorridos-realizados-2020.zip
ya descargado ./datasets/recorridos-realizados-2021.zip
done


In [16]:
bicicletas["recorridos"]["2021"]["df"].info()
bicicletas["recorridos"]["2020"]["df"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328726 entries, 0 to 1328725
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1328726 non-null  Int64  
 1   estado_cerrado             1328726 non-null  object 
 2   duracion_recorrido         1328726 non-null  int64  
 3   id_estacion_origen         1328726 non-null  int64  
 4   fecha_origen_recorrido     1328726 non-null  object 
 5   nombre_estacion_origen     1328726 non-null  object 
 6   fecha_destino_recorrido    1328726 non-null  object 
 7   id_estacion_destino        1328723 non-null  float64
 8   nombre_estacion_destino    1328723 non-null  object 
 9   id_usuario                 1328726 non-null  int64  
 10  cilicsta_tipo              1328726 non-null  object 
 11  bicicleta_modelo           1328726 non-null  object 
 12  direccion_estacion_origen  1328726 non-null  object 
dtypes: Int64(1),

In [5]:
# Concatenar recorridos 2020 2021
frames = []
for r in bicicletas["recorridos"]:
    frames.append( bicicletas["recorridos"][r]["df"] )
bicicletas["recorridos_df"] = pd.concat( frames )

In [17]:
bicicletas["recorridos_df"] 

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido
593758,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13
687904,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29
1026453,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46
1876322,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54
1126779,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19
...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,129 - Velasco,2021-06-02 16:09:32,158.0,158 - VILLARROEL,2021-06-02 16:19:23
3,11204920,400,713938,117,117 - HUMBERTO 1°,2021-06-02 16:09:34,175.0,147 - Constitución,2021-06-02 16:16:14
2,11204928,299,690073,232,252 - Tamborini,2021-06-02 16:10:08,386.0,277 - Coghlan,2021-06-02 16:15:07
1,11204938,406,348428,387,278 - Donato Alvarez,2021-06-02 16:11:01,262.0,349 - Plaza Saenz Peña,2021-06-02 16:17:47


In [6]:
# Fill nan en id y destinos
# bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].interpolate( method='linear' ) # no funco
bicicletas["recorridos_df"] = bicicletas["recorridos_df"].sort_values( by=['fecha_origen_recorrido'] )
bicicletas["recorridos_df"]["id"] = bicicletas["recorridos_df"]["id"].fillna(0)
bicicletas["recorridos_df"]['nombre_estacion_destino'] = bicicletas["recorridos_df"]['nombre_estacion_destino'].fillna('0 - null')

In [7]:
# Filtro y ordeno columnas que quedan
orden_filtro = [
    'id',
    'duracion_recorrido',
#     'estado_cerrado',
    'id_usuario',
#     'cilicsta_tipo',
#     'bicicleta_modelo',
    'id_estacion_origen',
    'nombre_estacion_origen',       
#     'direccion_estacion_origen',
    'fecha_origen_recorrido',       
#     'long_estacion_origen',     
#     'lat_estacion_origen',
    'id_estacion_destino',         
    'nombre_estacion_destino',    
#     'direccion_estacion_destino',
    'fecha_destino_recorrido',      
#     'long_estacion_destino',       
#     'lat_estacion_destino', 
#     'periodo',                       
]
bicicletas["recorridos_df"] = bicicletas["recorridos_df"][orden_filtro]

In [19]:
bicicletas["recorridos_df"]

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido
593758,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13
687904,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29
1026453,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46
1876322,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54
1126779,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19
...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,129 - Velasco,2021-06-02 16:09:32,158.0,158 - VILLARROEL,2021-06-02 16:19:23
3,11204920,400,713938,117,117 - HUMBERTO 1°,2021-06-02 16:09:34,175.0,147 - Constitución,2021-06-02 16:16:14
2,11204928,299,690073,232,252 - Tamborini,2021-06-02 16:10:08,386.0,277 - Coghlan,2021-06-02 16:15:07
1,11204938,406,348428,387,278 - Donato Alvarez,2021-06-02 16:11:01,262.0,349 - Plaza Saenz Peña,2021-06-02 16:17:47


### Procesar fecha

Parsear object datetime a dia, mes, año

In [8]:
# bicis_recorridos_2020_df['fecha_origen_recorrido']
formato = '%Y%m%d %H:%M:%S'
bicicletas["recorridos_df"]['start_datetime'] = pd.to_datetime(
    bicicletas["recorridos_df"]['fecha_origen_recorrido'], 
    format = formato
)
# bicicletas["recorridos_df"]['start_date'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y-%m-%d')
bicicletas["recorridos_df"]['anio'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%Y')
bicicletas["recorridos_df"]['nmes'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%m')
bicicletas["recorridos_df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b').str.lower()
bicicletas["recorridos_df"]['ndia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%j')
bicicletas["recorridos_df"]['dia']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%a').str.lower()
bicicletas["recorridos_df"]['wdia'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%w')
bicicletas["recorridos_df"]['hora'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%H')
bicicletas["recorridos_df"]['mins'] = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%M')
bicicletas["recorridos_df"]

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins
593758,0,6563,506009,30,030 - Peña,2020-01-01 00:02:50,30.0,030 - Peña,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,01,jan,001,wed,3,00,02
687904,0,1150,116847,206,159 - Fitz Roy & Gorriti,2020-01-01 00:04:19,207.0,123 - Armenia,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,01,jan,001,wed,3,00,04
1026453,0,383,259973,308,251 - Solar de la Abadía,2020-01-01 00:05:23,289.0,255 - BARRANCAS DE BELGRANO,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,01,jan,001,wed,3,00,05
1876322,0,72,354594,92,092 - Salcedo,2020-01-01 00:05:42,92.0,092 - Salcedo,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,01,jan,001,wed,3,00,05
1126779,0,769,222181,41,041 - Urquiza y Rondeau,2020-01-01 00:07:30,41.0,041 - Urquiza y Rondeau,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,01,jan,001,wed,3,00,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,129 - Velasco,2021-06-02 16:09:32,158.0,158 - VILLARROEL,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,06,jun,153,wed,3,16,09
3,11204920,400,713938,117,117 - HUMBERTO 1°,2021-06-02 16:09:34,175.0,147 - Constitución,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,06,jun,153,wed,3,16,09
2,11204928,299,690073,232,252 - Tamborini,2021-06-02 16:10:08,386.0,277 - Coghlan,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,06,jun,153,wed,3,16,10
1,11204938,406,348428,387,278 - Donato Alvarez,2021-06-02 16:11:01,262.0,349 - Plaza Saenz Peña,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,06,jun,153,wed,3,16,11


In [9]:
# Separar las columnas nombre de estacion de origen y nombre de estacion de destino 
# Tienen la info correcta del id de las estaciones

# Use vectoried str.split with expand = True:
bicicletas["recorridos_df"][
    ['id_estacion_incio','nombre_estacion_origen']
] = bicicletas["recorridos_df"]['nombre_estacion_origen'].str.split( ' - ', expand = True )

# Aca divido la columna Nombre de estacion de fin de viaje porque contiene dos campos 
# el Id y el nombre de la estacion de inicio de viaje.
# registros Nan ya pàsados a '0 - null'
bicicletas["recorridos_df"][
    ['id_estacion_destino','nombre_estacion_destino']
] = bicicletas["recorridos_df"]['nombre_estacion_destino'].str.split( ' - ', expand = True )
# TODO: sacar los leading zeros

# bicicletas["recorridos_df"]["id_estacion_incio"] = bicicletas["recorridos_df"]["id_estacion_incio"].astype('Int64')
bicicletas["recorridos_df"]

,id,duracion_recorrido,id_usuario,id_estacion_origen,nombre_estacion_origen,fecha_origen_recorrido,id_estacion_destino,nombre_estacion_destino,fecha_destino_recorrido,start_datetime,anio,nmes,mes,ndia,dia,wdia,hora,mins,id_estacion_incio
593758,0,6563,506009,30,Peña,2020-01-01 00:02:50,030,Peña,2020-01-01 01:52:13,2020-01-01 00:02:50,2020,01,jan,001,wed,3,00,02,030
687904,0,1150,116847,206,Fitz Roy & Gorriti,2020-01-01 00:04:19,123,Armenia,2020-01-01 00:23:29,2020-01-01 00:04:19,2020,01,jan,001,wed,3,00,04,159
1026453,0,383,259973,308,Solar de la Abadía,2020-01-01 00:05:23,255,BARRANCAS DE BELGRANO,2020-01-01 00:11:46,2020-01-01 00:05:23,2020,01,jan,001,wed,3,00,05,251
1876322,0,72,354594,92,Salcedo,2020-01-01 00:05:42,092,Salcedo,2020-01-01 00:06:54,2020-01-01 00:05:42,2020,01,jan,001,wed,3,00,05,092
1126779,0,769,222181,41,Urquiza y Rondeau,2020-01-01 00:07:30,041,Urquiza y Rondeau,2020-01-01 00:20:19,2020-01-01 00:07:30,2020,01,jan,001,wed,3,00,07,041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,11204919,591,35926,212,Velasco,2021-06-02 16:09:32,158,VILLARROEL,2021-06-02 16:19:23,2021-06-02 16:09:32,2021,06,jun,153,wed,3,16,09,129
3,11204920,400,713938,117,HUMBERTO 1°,2021-06-02 16:09:34,147,Constitución,2021-06-02 16:16:14,2021-06-02 16:09:34,2021,06,jun,153,wed,3,16,09,117
2,11204928,299,690073,232,Tamborini,2021-06-02 16:10:08,277,Coghlan,2021-06-02 16:15:07,2021-06-02 16:10:08,2021,06,jun,153,wed,3,16,10,252
1,11204938,406,348428,387,Donato Alvarez,2021-06-02 16:11:01,349,Plaza Saenz Peña,2021-06-02 16:17:47,2021-06-02 16:11:01,2021,06,jun,153,wed,3,16,11,278


# Exportar CSV
- Download
- Upload to cloud

In [ ]:
bicicletas["recorridos_df"].to_csv(
    'recorridos_2020-2021.zip',
    index = False,
    compression = {
        method =  'zip',
        archive_name = 'recorridos_2020-2021.csv'
    } 
)

In [ ]:
from google.colab import files
files.download('recorridos_2020-2021.zip')

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
bicicletas["recorridos_df"].to_csv('/drive/My Drive/Datasets-bicing/recorridos_2020-2021.csv')

## Bajar el ultimo
Bajar el dataset limpio desde drive

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = '17liAPD1lyj2wT-Hcbh4ed9-Cn4peoyIq'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos_2020-2021.csv') 
bicicletas["recorridos_df"] = pd.read_csv('recorridos_2020-2021.csv')
bicicletas["recorridos_df"].head()

### Agrego datos del clima (queda por hacer)

In [ ]:
# Fetch from drive
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(
    ['DATE','PRCP','TAVG']
)
weather_data_df.isnull().sum( axis = 0 )
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace( np.nan, 0 )
weather_data_df.head()

In [ ]:
bicicletas["recorridos_clima_df"] = pd.merge(
    left = bicicletas["recorridos_df"],
    right = weather_data_df,
    left_on = 'start_datetime',
    right_on = 'DATE'
)

In [ ]:
bicicletas["recorridos_clima_df"].head()

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
bicicletas["recorridos_clima_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos-clima-2020-2021.csv'
)